In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!pip install -q pydrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
!pip install transformers

In [ ]:
!pip install simpletransformers

In [ ]:
project_path='/content/drive/My Drive/Fake News/'

In [ ]:
import numpy as np
import pandas as pd

df_train=pd.read_csv(project_path+'train.csv')

texts_train=[]
texts_train=df_train['Statement']

label_train=df_train['Label']


X_train=texts_train.astype(str).values
X_train=np.reshape(X_train,(-1,1))


label_train=label_train.astype(int).values
y_train=np.reshape(label_train,(-1,1))

In [ ]:
df_test=pd.read_csv(project_path+'test.csv')

texts_test=[]
texts_test=df_test['Statement']
label_test=df_test['Label']


X_test=texts_test.astype(str).values
X_test=np.reshape(X_test,(-1,1))


label_test=label_test.astype(int).values
y_test=np.reshape(label_test,(-1,1))

In [ ]:
sentences = X_train[:,0]
print(sentences[0])
labels = y_train[:, 0]
#labels = y_train
print(labels[0])

In [ ]:
print('Max sentence length: ', max([len(sen.split(' ')) for sen in sentences]))

In [ ]:
VALIDATION_SPLIT = 0.2

nb_validation_samples = int(VALIDATION_SPLIT * sentences.shape[0])

x_train = sentences[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = sentences[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print(x_train[0])
print(x_val[0])

In [ ]:
import pandas as pd
ll=[]
for i in range(len(x_train)):
  ll.append([x_train[i],y_train[i]])

train_df = pd.DataFrame(ll,columns=['text','labels'])


ll=[]
for i in range(len(x_val)):
  ll.append([x_val[i],y_val[i]])

val_df = pd.DataFrame(ll,columns=['text','labels'])

In [ ]:
sentences_test = X_test[:,0]
print(sentences_test[0])
labels_test = y_test[:, 0]
#labels = y_train
print(labels_test[0])

In [ ]:
ll=[]
for i in range(len(sentences_test)):
  ll.append([sentences_test[i],labels_test[i]])

test_df = pd.DataFrame(ll,columns=['text','labels'])

In [ ]:
print('Training sample: ' + str(len(train_df['text'])))
print('Validation sample: ' + str(len(val_df['text'])))
print('Testing sample: ' + str(len(test_df['text'])))

In [ ]:
model_type = "bert"
model_name = "bert-base-cased"

train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": "/content/drive/My Drive/Fake News/Transformer/output/"+model_type,
    "best_model_dir": "/content/drive/My Drive/Fake News/Transformer/output/"+model_type+"/best_model",
    "use_early_stopping": False,
    "early_stopping_delta": 0.0,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize" : True,
    "early_stopping_patience" : 2,
    "evaluate_during_training": True,
    "max_seq_length": 512,
    "num_train_epochs": 10,
    "evaluate_during_training_steps": 64,
    "wandb_project": "Fake News Liar_1 Bert",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "train_batch_size": 64,
    "eval_batch_size": 64,
    "evaluate_during_training_verbose" : True
}

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
#import pandas as pd

train_args["max_seq_length"] = 300
train_args["train_batch_size"] = 32
train_args["gradient_accumulation_steps"] = 1
train_args["evaluate_during_training"] = True
train_args["evaluate_during_training_steps"] = 256
train_args["num_train_epochs"] = 10
train_args["use_early_stopping"] = True


# Create a ClassificationModel
model = ClassificationModel(model_type, model_name, args=train_args)

# Train the model
model.train_model(train_df,eval_df = val_df)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test_df)

In [ ]:
predictions, raw_outputs = model.predict(test_df['text'])

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(test_df['labels'],predictions))

In [ ]:
import pickle
with open(project_path+'Predictions/pickle_Pred_Bert_1_Liar.pickle', 'wb') as f:
  pickle.dump((test_df['labels'],predictions),f)


In [ ]:
import pickle
with open(project_path+'Predictions/pickle_Pred_Bert_1_Liar.pickle', 'rb') as f:
  y_true, y_pred = pickle.load(f)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_true, y_pred))